## 2020~2022년 : https://www.data.go.kr/data/15067528/fileData.do
## 2018~2019년 : https://data.seoul.go.kr/dataList/OA-15067/S/1/datasetView.do#

In [1]:
# pip install geopy

In [2]:
import numpy as np
import pandas as pd
import os
import time
import googlemaps
from geopy.geocoders import Nominatim
import requests, json, pprint

In [4]:
df=pd.read_csv('C:/acorn/midproject/bus_stop/raw_data/2020년_전국버스정류장 위치정보.csv', encoding='cp949')

C:\Users\khg08\AppData\Local\Temp\ipykernel_38788\1818219887.py:1: DtypeWarning: Columns (5,8,11,15,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('C:/acorn/midproject/bus_stop/raw_data/2020년_전국버스정류장 위치정보.csv', encoding='cp949')


In [5]:
df.head()

,NODE_ID,NODE_NM,NODE_TP,CENTER_YN,ADMIN_NM,REMARK,NODE_CL,NODE_SUB_TP,NODE_ENM,GPS_LATI,...,DISTRICT_ID,COLLECTD_TIME,ORG_ID,ADMIN_ID,OFFSET,DIRECTION,DOWNTOWN_YN,NODE_MOBILE_ID,CITY_CODE,CITY_NAME
0,TSB252000435,행정,1.0,N,강릉시,NaN,NaN,NaN,Haengjeong,37.852718,...,NaN,2020-09-24 04:00:04,NaN,252.0,NaN,NaN,NaN,252000435.0,32030,강릉시
1,TSB252000436,행정,1.0,N,강릉시,NaN,NaN,NaN,Haengjeong,37.852776,...,NaN,2020-09-24 04:00:04,NaN,252.0,NaN,NaN,NaN,252000436.0,32030,강릉시
2,TSB252000437,동양시멘트입구,1.0,N,강릉시,NaN,NaN,NaN,Dongyangsimenteuipgu,37.850532,...,NaN,2020-09-24 04:00:04,NaN,252.0,NaN,NaN,NaN,252000437.0,32030,강릉시
3,TSB252000438,동양시멘트입구,1.0,N,강릉시,NaN,NaN,NaN,Dongyangsimenteuipgu,37.850634,...,NaN,2020-09-24 04:00:04,NaN,252.0,NaN,NaN,NaN,252000438.0,32030,강릉시
4,TSB252000439,송림2리경로당,1.0,N,강릉시,NaN,NaN,NaN,Songnim2Rigyeongnodang,37.852018,...,NaN,2020-09-24 04:00:04,NaN,252.0,NaN,NaN,NaN,252000439.0,32030,강릉시


In [6]:
df.columns

Index(['NODE_ID', 'NODE_NM', 'NODE_TP', 'CENTER_YN', 'ADMIN_NM', 'REMARK',
       'NODE_CL', 'NODE_SUB_TP', 'NODE_ENM', 'GPS_LATI', 'GPS_LONG', 'CITY_CD',
       'DISTRICT_ID', 'COLLECTD_TIME', 'ORG_ID', 'ADMIN_ID', 'OFFSET',
       'DIRECTION', 'DOWNTOWN_YN', 'NODE_MOBILE_ID', 'CITY_CODE', 'CITY_NAME'],
      dtype='object')

In [7]:
df['CITY_NAME'].unique()

array(['강릉시', '동해시', '태백시', '속초시', '삼척시', '홍천군', '진천군', '음성군', '단양군',
       '논산시', '부여군', '영동군', '영월군', '정선군', '철원군', '화천군', '양구군', '인제군',
       '고성군', '양양군', '평창군', '정읍시', '진안군', '무주군', '임실군', '순창군', '고창군',
       '부안군', '장수군', '당진시', '익산시', '상주시', '고흥군', '강진군', '영광군', '영주시',
       '안동시', '아산시', '보은군', '창원시', '천안시', '칠곡군', '춘천시', '충주시', '대구광역시',
       '청주시', '대전광역시', '거제시', '김천시', '김해시', '공주시', '구미시', '광주광역시', '광양시',
       '군산시', '경주시', '경산시', '인천광역시', '함평군', '제주도', '전주시', '진주시', '제천시',
       '옥천군', '포항시', '세종특별시', '남원시', '밀양시', '목포시', '순천시', '통영시', '서산시',
       '울산광역시', '원주시', '여수시', '서울특별시', '양산시', '영천시', '수원시', '성남시', '의정부시',
       '광주시', '안산시', '고양시', '과천시', '구리시', '남양주시', '동두천시', '안양시', '부천시',
       '광명시', '평택시', '용인시', '파주시', '이천시', '안성시', '오산시', '시흥시', '군포시',
       '의왕시', '하남시', '양주시', '포천시', '여주군', '연천군', '김포시', '화성시', '양평군',
       '가평군'], dtype=object)

In [8]:
df.rename(columns={'CITY_NAME':'도시명'}, inplace=True)

In [9]:
df.head()

,NODE_ID,NODE_NM,NODE_TP,CENTER_YN,ADMIN_NM,REMARK,NODE_CL,NODE_SUB_TP,NODE_ENM,GPS_LATI,...,DISTRICT_ID,COLLECTD_TIME,ORG_ID,ADMIN_ID,OFFSET,DIRECTION,DOWNTOWN_YN,NODE_MOBILE_ID,CITY_CODE,도시명
0,TSB252000435,행정,1.0,N,강릉시,NaN,NaN,NaN,Haengjeong,37.852718,...,NaN,2020-09-24 04:00:04,NaN,252.0,NaN,NaN,NaN,252000435.0,32030,강릉시
1,TSB252000436,행정,1.0,N,강릉시,NaN,NaN,NaN,Haengjeong,37.852776,...,NaN,2020-09-24 04:00:04,NaN,252.0,NaN,NaN,NaN,252000436.0,32030,강릉시
2,TSB252000437,동양시멘트입구,1.0,N,강릉시,NaN,NaN,NaN,Dongyangsimenteuipgu,37.850532,...,NaN,2020-09-24 04:00:04,NaN,252.0,NaN,NaN,NaN,252000437.0,32030,강릉시
3,TSB252000438,동양시멘트입구,1.0,N,강릉시,NaN,NaN,NaN,Dongyangsimenteuipgu,37.850634,...,NaN,2020-09-24 04:00:04,NaN,252.0,NaN,NaN,NaN,252000438.0,32030,강릉시
4,TSB252000439,송림2리경로당,1.0,N,강릉시,NaN,NaN,NaN,Songnim2Rigyeongnodang,37.852018,...,NaN,2020-09-24 04:00:04,NaN,252.0,NaN,NaN,NaN,252000439.0,32030,강릉시


In [10]:
df = df[df['도시명']=='서울특별시']

In [11]:
df

,NODE_ID,NODE_NM,NODE_TP,CENTER_YN,ADMIN_NM,REMARK,NODE_CL,NODE_SUB_TP,NODE_ENM,GPS_LATI,...,DISTRICT_ID,COLLECTD_TIME,ORG_ID,ADMIN_ID,OFFSET,DIRECTION,DOWNTOWN_YN,NODE_MOBILE_ID,CITY_CODE,도시명
121632,SEB204000339,태평고개,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.444342,...,NaN,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시
121633,SEB204000108,태평3동행정복지센터,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.445649,...,NaN,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시
121634,SEB205000030,자동차검사소.산성아파트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.441654,...,NaN,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시
121635,SEB206000217,분당중앙공원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.373525,...,NaN,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시
121636,SEB206000314,아름마을.이매고교.효성선경아파,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.395315,...,NaN,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,7303.0,11,서울특별시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151851,SEB227000549,현대재활용,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.541307,...,NaN,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,28352.0,11,서울특별시
151852,SEB227000625,감천슈퍼,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.520367,...,NaN,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,28432.0,11,서울특별시
151853,SEB235000203,소놀이굿전수회관,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.793490,...,NaN,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,39335.0,11,서울특별시
151854,SEB235000773,중말,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.794777,...,NaN,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시


In [12]:
df.reset_index(inplace=True)

In [13]:
del df['index']

In [14]:
df.head()

,NODE_ID,NODE_NM,NODE_TP,CENTER_YN,ADMIN_NM,REMARK,NODE_CL,NODE_SUB_TP,NODE_ENM,GPS_LATI,...,DISTRICT_ID,COLLECTD_TIME,ORG_ID,ADMIN_ID,OFFSET,DIRECTION,DOWNTOWN_YN,NODE_MOBILE_ID,CITY_CODE,도시명
0,SEB204000339,태평고개,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.444342,...,NaN,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시
1,SEB204000108,태평3동행정복지센터,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.445649,...,NaN,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시
2,SEB205000030,자동차검사소.산성아파트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.441654,...,NaN,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시
3,SEB206000217,분당중앙공원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.373525,...,NaN,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시
4,SEB206000314,아름마을.이매고교.효성선경아파,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.395315,...,NaN,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,7303.0,11,서울특별시


In [15]:
df.rename(columns={'GPS_LATI':'위도', 'GPS_LONG':'경도'}, inplace=True)

In [16]:
def get_address(lat, lng):
    
    try : 
        url = "https://dapi.kakao.com/v2/local/geo/coord2address.json?x="+lng+"&y="+lat
        headers = {"Authorization": "KakaoAK 3f4e3aa092149cf60bd447f49294d7c2"}
        api_json = requests.get(url, headers=headers)

        full_address = json.loads(api_json.text)

        return full_address['documents'][0]['address']['region_2depth_name']
    
    except : 
        guname = 'error'

In [ ]:
gulist=[]
for i in range(len(df)):
    guname = get_address(str(df['위도'][i]), str(df['경도'][i]))
    gulist.append(guname)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204


In [26]:
len(gulist)

22439

In [27]:
gulist

['성남시 수정구',
 '성남시 수정구',
 '성남시 중원구',
 '성남시 분당구',
 '성남시 분당구',
 '성남시 분당구',
 '성남시 분당구',
 '성남시 분당구',
 '성남시 분당구',
 '성남시 분당구',
 '성남시 분당구',
 '성남시 분당구',
 '성남시 분당구',
 '성남시 분당구',
 '성남시 분당구',
 '중구',
 '성남시 분당구',
 '성남시 수정구',
 '하남시',
 '성남시 수정구',
 '광주시',
 '광주시',
 '성남시 분당구',
 '성남시 분당구',
 '성남시 분당구',
 '광주시',
 '용인시 처인구',
 '용인시 처인구',
 '용인시 처인구',
 '용인시 처인구',
 '용인시 처인구',
 '성남시 분당구',
 '성남시 분당구',
 '성남시 수정구',
 '성남시 분당구',
 '중랑구',
 '용인시 수지구',
 '용인시 수지구',
 '성남시 분당구',
 '성남시 분당구',
 '성남시 분당구',
 '용인시 수지구',
 '용인시 수지구',
 '의정부시',
 '의정부시',
 '의정부시',
 '의정부시',
 '의정부시',
 '의정부시',
 '의정부시',
 '의정부시',
 '의정부시',
 '구리시',
 '구리시',
 '구리시',
 '광진구',
 '광진구',
 '광진구',
 '구리시',
 '구리시',
 '구리시',
 '구리시',
 '구리시',
 '남양주시',
 '의정부시',
 '양주시',
 '양주시',
 '양주시',
 '양주시',
 '양주시',
 '양주시',
 '양주시',
 '양주시',
 '양주시',
 '의정부시',
 '의정부시',
 '의정부시',
 '의정부시',
 '의정부시',
 '양주시',
 '양주시',
 '양주시',
 '동두천시',
 '양주시',
 '양주시',
 '양주시',
 '양주시',
 '양주시',
 '의정부시',
 '의정부시',
 '양주시',
 '양주시',
 '동두천시',
 '동두천시',
 '동두천시',
 '양주시',
 '양주시',
 '양주시',
 '용산구',
 '중구',
 '남양주시',
 '남양주시',
 '남양주시',
 '중랑구

In [28]:
df['gu']=gulist

C:\Users\khg08\AppData\Local\Temp\ipykernel_28916\2169215124.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gu']=gulist


In [29]:
df.head()

,NODE_ID,NODE_NM,NODE_TP,CENTER_YN,ADMIN_NM,REMARK,NODE_CL,NODE_SUB_TP,NODE_ENM,위도,...,COLLECTD_TIME,ORG_ID,ADMIN_ID,OFFSET,DIRECTION,DOWNTOWN_YN,NODE_MOBILE_ID,CITY_CODE,도시명,gu
0,SEB204000339,태평고개,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.444342,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시,성남시 수정구
1,SEB204000108,태평3동행정복지센터,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.445649,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시,성남시 수정구
2,SEB205000030,자동차검사소.산성아파트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.441654,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시,성남시 중원구
3,SEB206000217,분당중앙공원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.373525,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시,성남시 분당구
4,SEB206000314,아름마을.이매고교.효성선경아파,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.395315,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,7303.0,11,서울특별시,성남시 분당구


In [30]:
len(df)

22439

In [31]:
df.isnull().sum()

NODE_ID               0
NODE_NM               0
NODE_TP           22439
CENTER_YN         22439
ADMIN_NM          22439
REMARK            22439
NODE_CL           22439
NODE_SUB_TP       22439
NODE_ENM          22439
위도                    0
경도                    0
CITY_CD           22439
DISTRICT_ID       22439
COLLECTD_TIME         0
ORG_ID            22439
ADMIN_ID          22439
OFFSET            22439
DIRECTION         22439
DOWNTOWN_YN       22439
NODE_MOBILE_ID        0
CITY_CODE             0
도시명                   0
gu                   57
dtype: int64

In [32]:
indices = df.index[df['gu'].isnull()].tolist()
indices

[255,
 576,
 1810,
 2907,
 2986,
 3244,
 4012,
 5155,
 5297,
 5303,
 6140,
 6245,
 6466,
 6826,
 6921,
 6965,
 7664,
 7849,
 7854,
 7885,
 8311,
 8459,
 8853,
 8870,
 9020,
 10372,
 12485,
 13470,
 14181,
 14632,
 15524,
 15619,
 15868,
 16395,
 16513,
 16812,
 16935,
 17260,
 17385,
 18048,
 18969,
 18991,
 19003,
 19267,
 19363,
 19427,
 19522,
 19523,
 19551,
 19567,
 19651,
 19921,
 20720,
 21117,
 21202,
 21497,
 22434]

In [33]:
len(indices)

57

In [34]:
for i in indices:
    
    lat = str(df['위도'][i])
    lng = str(df['경도'][i])
    
    url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x="+lng+"&y="+lat
    headers = {"Authorization": "KakaoAK 3f4e3aa092149cf60bd447f49294d7c2"}
    api_json = requests.get(url, headers=headers)
    full_address = json.loads(api_json.text)
    
    df['gu'][i] = full_address['documents'][0]['region_2depth_name'] 
    print(i)

C:\Users\khg08\AppData\Local\Temp\ipykernel_28916\67907269.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gu'][i] = full_address['documents'][0]['region_2depth_name']


255
576
1810
2907
2986
3244
4012
5155
5297
5303
6140
6245
6466
6826
6921
6965
7664
7849
7854
7885
8311
8459
8853
8870
9020
10372
12485
13470
14181
14632
15524
15619
15868
16395
16513
16812
16935
17260
17385
18048
18969
18991
19003
19267
19363
19427
19522
19523
19551
19567
19651
19921
20720
21117
21202
21497
22434


In [35]:
indices_after = df.index[df['gu'].isnull()].tolist()
indices_after

[]

In [36]:
df.iloc[indices]

,NODE_ID,NODE_NM,NODE_TP,CENTER_YN,ADMIN_NM,REMARK,NODE_CL,NODE_SUB_TP,NODE_ENM,위도,...,COLLECTD_TIME,ORG_ID,ADMIN_ID,OFFSET,DIRECTION,DOWNTOWN_YN,NODE_MOBILE_ID,CITY_CODE,도시명,gu
255,SEB277102365,고강차고지(경유),NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.536016,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,78629.0,11,서울특별시,부천시
576,SEB111000012,구파발역입구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.637116,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,12012.0,11,서울특별시,은평구
1810,SEB118000207,대방역,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.513230,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,19295.0,11,서울특별시,영등포구
2907,SEB219000222,고양자유학교앞,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.721714,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시,고양시 일산동구
2986,SEB107900241,정든마을,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.609505,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,8910.0,11,서울특별시,성북구
3244,SEB234000344,묵방리,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.344000,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시,광주시
4012,SEB165000736,에코메트로10단지,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.387526,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시,남동구
5155,SEB165000728,에코메트로10단지,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.387936,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시,남동구
5297,SEB111000345,입곡삼거리,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.649733,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,12440.0,11,서울특별시,은평구
5303,SEB111000149,진관초교관음사입구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.647773,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,12238.0,11,서울특별시,은평구


In [37]:
df

,NODE_ID,NODE_NM,NODE_TP,CENTER_YN,ADMIN_NM,REMARK,NODE_CL,NODE_SUB_TP,NODE_ENM,위도,...,COLLECTD_TIME,ORG_ID,ADMIN_ID,OFFSET,DIRECTION,DOWNTOWN_YN,NODE_MOBILE_ID,CITY_CODE,도시명,gu
0,SEB204000339,태평고개,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.444342,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시,성남시 수정구
1,SEB204000108,태평3동행정복지센터,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.445649,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시,성남시 수정구
2,SEB205000030,자동차검사소.산성아파트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.441654,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시,성남시 중원구
3,SEB206000217,분당중앙공원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.373525,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시,성남시 분당구
4,SEB206000314,아름마을.이매고교.효성선경아파,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.395315,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,7303.0,11,서울특별시,성남시 분당구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22434,SEB227000549,현대재활용,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.541307,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,28352.0,11,서울특별시,하남시
22435,SEB227000625,감천슈퍼,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.520367,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,28432.0,11,서울특별시,하남시
22436,SEB235000203,소놀이굿전수회관,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.793490,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,39335.0,11,서울특별시,양주시
22437,SEB235000773,중말,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.794777,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,0.0,11,서울특별시,양주시


In [38]:
df.gu.unique()

array(['성남시 수정구', '성남시 중원구', '성남시 분당구', '중구', '하남시', '광주시', '용인시 처인구',
       '중랑구', '용인시 수지구', '의정부시', '구리시', '광진구', '남양주시', '양주시', '동두천시',
       '용산구', '성북구', '마포구', '영등포구', '동대문구', '서대문구', '은평구', '송파구', '강남구',
       '종로구', '성동구', '강동구', '서초구', '도봉구', '포천시', '노원구', '군포시', '안양시 만안구',
       '안양시 동안구', '의왕시', '부천시', '강서구', '시흥시', '과천시', '관악구', '동작구', '구로구',
       '광명시', '금천구', '양천구', '강북구', '파주시', '고양시 일산동구', '고양시 일산서구',
       '고양시 덕양구', '부평구', '계양구', '미추홀구', '연수구', '안산시 상록구', '안산시 단원구',
       '화성시', '김포시', '가평군', '용인시 기흥구', '수원시 영통구', '서구', '강화군', '수원시 장안구',
       '수원시 권선구', '수원시 팔달구', '오산시', '평택시', '연천군', '양평군', '안성시', '여주시',
       '천안시 서북구', '이천시', '남동구', '진천군', '음성군', '철원군', '천안시 동남구', '아산시'],
      dtype=object)

In [39]:
df.to_csv('2020_seoulgg_gu.csv')

In [40]:
seoulgulist = [
    '강남구',
    '강동구',
    '강북구',
    '강서구',
    '관악구',
    '광진구',
    '구로구',
    '금천구',
    '노원구',
    '도봉구',
    '동대문구',
    '동작구',
    '마포구',
    '서대문구',
    '서초구',
    '성동구',
    '성북구',
    '송파구',
    '양천구',
    '영등포구',
    '용산구',
    '은평구',
    '종로구',
    '중구',
    '중랑구'
]

In [41]:
seoulgu = pd.DataFrame(seoulgulist, columns=['gu'])

In [42]:
seoulgu

,gu
0,강남구
1,강동구
2,강북구
3,강서구
4,관악구
5,광진구
6,구로구
7,금천구
8,노원구
9,도봉구


In [43]:
result = pd.merge(df, seoulgu)
result

,NODE_ID,NODE_NM,NODE_TP,CENTER_YN,ADMIN_NM,REMARK,NODE_CL,NODE_SUB_TP,NODE_ENM,위도,...,COLLECTD_TIME,ORG_ID,ADMIN_ID,OFFSET,DIRECTION,DOWNTOWN_YN,NODE_MOBILE_ID,CITY_CODE,도시명,gu
0,SEB101000038,시청.서소문청사,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.563525,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,2135.0,11,서울특별시,중구
1,SEB101000130,동대문역사문화공원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.565411,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,2235.0,11,서울특별시,중구
2,SEB101000011,"서울의료보건고등학교,서울역서부",NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.556552,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,2103.0,11,서울특별시,중구
3,SEB101000010,손기정체육공원입구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.554446,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,2102.0,11,서울특별시,중구
4,SEB101000263,서울스퀘어앞,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.555342,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,2282.0,11,서울특별시,중구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11005,SEB108000145,번동해모로아파트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.626624,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,9233.0,11,서울특별시,강북구
11006,SEB108000159,수유3동우체국,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.640108,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,9247.0,11,서울특별시,강북구
11007,SEB108000367,삼각산119안전센터,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.619853,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,9307.0,11,서울특별시,강북구
11008,SEB108000200,삼각산동삼각산아이원아파트정문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.616178,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,9295.0,11,서울특별시,강북구


In [44]:
len(df)-len(result)

11429

In [45]:
result.gu.unique()

array(['중구', '중랑구', '광진구', '용산구', '성북구', '마포구', '영등포구', '동대문구', '서대문구',
       '은평구', '송파구', '강남구', '종로구', '성동구', '강동구', '서초구', '도봉구', '노원구',
       '강서구', '관악구', '동작구', '구로구', '금천구', '양천구', '강북구'], dtype=object)

In [46]:
result.to_csv('bus_2020.csv')

#### 구별 정류장개수 구하기

In [47]:
result

,NODE_ID,NODE_NM,NODE_TP,CENTER_YN,ADMIN_NM,REMARK,NODE_CL,NODE_SUB_TP,NODE_ENM,위도,...,COLLECTD_TIME,ORG_ID,ADMIN_ID,OFFSET,DIRECTION,DOWNTOWN_YN,NODE_MOBILE_ID,CITY_CODE,도시명,gu
0,SEB101000038,시청.서소문청사,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.563525,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,2135.0,11,서울특별시,중구
1,SEB101000130,동대문역사문화공원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.565411,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,2235.0,11,서울특별시,중구
2,SEB101000011,"서울의료보건고등학교,서울역서부",NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.556552,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,2103.0,11,서울특별시,중구
3,SEB101000010,손기정체육공원입구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.554446,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,2102.0,11,서울특별시,중구
4,SEB101000263,서울스퀘어앞,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.555342,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,2282.0,11,서울특별시,중구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11005,SEB108000145,번동해모로아파트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.626624,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,9233.0,11,서울특별시,강북구
11006,SEB108000159,수유3동우체국,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.640108,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,9247.0,11,서울특별시,강북구
11007,SEB108000367,삼각산119안전센터,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.619853,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,9307.0,11,서울특별시,강북구
11008,SEB108000200,삼각산동삼각산아이원아파트정문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.616178,...,2020-09-24 07:32:12,NaN,NaN,NaN,NaN,NaN,9295.0,11,서울특별시,강북구


In [48]:
result1=result.groupby('gu').count()

In [49]:
result1

,NODE_ID,NODE_NM,NODE_TP,CENTER_YN,ADMIN_NM,REMARK,NODE_CL,NODE_SUB_TP,NODE_ENM,위도,...,DISTRICT_ID,COLLECTD_TIME,ORG_ID,ADMIN_ID,OFFSET,DIRECTION,DOWNTOWN_YN,NODE_MOBILE_ID,CITY_CODE,도시명
gu,,,,,,,,,,,,,,,,,,,,,
강남구,545,545,0,0,0,0,0,0,0,545,...,0,545,0,0,0,0,0,545,545,545
강동구,383,383,0,0,0,0,0,0,0,383,...,0,383,0,0,0,0,0,383,383,383
강북구,415,415,0,0,0,0,0,0,0,415,...,0,415,0,0,0,0,0,415,415,415
강서구,611,611,0,0,0,0,0,0,0,611,...,0,611,0,0,0,0,0,611,611,611
관악구,470,470,0,0,0,0,0,0,0,470,...,0,470,0,0,0,0,0,470,470,470
광진구,288,288,0,0,0,0,0,0,0,288,...,0,288,0,0,0,0,0,288,288,288
구로구,510,510,0,0,0,0,0,0,0,510,...,0,510,0,0,0,0,0,510,510,510
금천구,357,357,0,0,0,0,0,0,0,357,...,0,357,0,0,0,0,0,357,357,357
노원구,538,538,0,0,0,0,0,0,0,538,...,0,538,0,0,0,0,0,538,538,538


In [51]:
result2 = pd.DataFrame(result1['NODE_NM'], index=result1.index)
result2

,NODE_NM
gu,
강남구,545
강동구,383
강북구,415
강서구,611
관악구,470
광진구,288
구로구,510
금천구,357
노원구,538


In [52]:
result2.columns=["정류장수"]

In [53]:
result2

,정류장수
gu,
강남구,545
강동구,383
강북구,415
강서구,611
관악구,470
광진구,288
구로구,510
금천구,357
노원구,538


In [54]:
result2['정류장수'].sum()

11010

In [55]:
result2.to_csv('2020_bus_count.csv')

In [123]:
len(result)

25